# Social Laws

This notebook shows a demonstration of social laws - a mechanism for coordinating multiple agents.

We will demonstrate social laws on a simple example of a 2x2 grid, with 2 agents on it. 
The locations are:
* nw (north west)
* sw (south west)
* ne (north east)
* se (south east)

Agent a1 want to get from nw to sw, and agent a2 wants to get from se to ne.

## Modeling Multi Agent Planning Problems for Social Laws

One of the basic concepts of social laws is _waiting_, and so we extend the ``MultiAgentProblem`` class to include waitfor annotations, and create a class called ``MultiAgentProblemWithWaitfor``

In [2]:
from unified_planning.shortcuts import *
from social_laws.ma_problem_waitfor import MultiAgentProblemWithWaitfor
from unified_planning.model.multi_agent import *
unified_planning.shortcuts.get_env().credits_stream = None

problem = MultiAgentProblemWithWaitfor("simple_example")

loc = UserType("loc")

# Environment     
connected = Fluent('connected', BoolType(), l1=loc, l2=loc)        
problem.ma_environment.add_fluent(connected, default_initial_value=False)

free = Fluent('free', BoolType(), l=loc)
problem.ma_environment.add_fluent(free, default_initial_value=True)

nw, ne, sw, se = Object("nw", loc), Object("ne", loc), Object("sw", loc), Object("se", loc)        
problem.add_objects([nw, ne, sw, se])
problem.set_initial_value(connected(nw, ne), True)
problem.set_initial_value(connected(nw, sw), True)
problem.set_initial_value(connected(ne, nw), True)
problem.set_initial_value(connected(ne, se), True)
problem.set_initial_value(connected(sw, se), True)
problem.set_initial_value(connected(sw, nw), True)
problem.set_initial_value(connected(se, sw), True)
problem.set_initial_value(connected(se, ne), True)


at = Fluent('at', BoolType(), l1=loc)

move = InstantaneousAction('move', l1=loc, l2=loc)
l1 = move.parameter('l1')
l2 = move.parameter('l2')
move.add_precondition(at(l1))
move.add_precondition(free(l2))
move.add_precondition(connected(l1,l2))
move.add_effect(at(l2),True)
move.add_effect(free(l2), False)
move.add_effect(at(l1), False)
move.add_effect(free(l1), True)    

agent1 = Agent("a1", problem)
problem.add_agent(agent1)
agent1.add_fluent(at, default_initial_value=False)
agent1.add_action(move)

agent2 = Agent("a2", problem)
problem.add_agent(agent2)
agent2.add_fluent(at, default_initial_value=False)
agent2.add_action(move)

problem.set_initial_value(Dot(agent1, at(nw)), True)
problem.set_initial_value(Dot(agent2, at(se)), True)
problem.set_initial_value(free(nw), False)
problem.set_initial_value(free(se), False)

problem.add_goal(Dot(agent1, at(sw)))
problem.add_goal(Dot(agent2, at(ne)))

print(problem)

AttributeError: module 'unified_planning.model' has no attribute 'State'

## Checking for Robustness

Social laws assume each agent plans individually, assuming the other agents do not perform any actions. A *robust* social law is one where this is guaranteed to produce correct plans.

To check whether our simple example is robust, we can use the ``SocialLawRobustnessChecker`` class.

``SocialLawRobustnessChecker`` first creates the single agent projection - a classical planning problem for each agent, assuming it is alone in the world, and solves them. If these are all solvable, it creates another classical planning problem which tries to get the agents to interfere with each other. If this problem is solvable, it provides a counter example to robustness. We can see the counterexample as actions from the original problems and as actions from the compilation - both have their uses.

In [ ]:
from social_laws.robustness_checker import SocialLawRobustnessChecker

slrc = SocialLawRobustnessChecker()

result = slrc.is_robust(problem)

print("status=", result.status)
print("counter example (original actions): ", result.counter_example_orig_actions)
print("counter example (compiled actions): ", result.counter_example)


status= SocialLawRobustnessStatus.NON_ROBUST_MULTI_AGENT_FAIL
counter example (original actions):  [move(nw, sw), move(se, sw), move(sw, se), move(se, ne)]
counter example (compiled actions):  [s_a1_move(nw, sw), f1_a2_move(se, sw), pc_a2_move(sw, se), pc_a2_move(se, ne), end_s_a1, end_f_a2_0]


## Applying Social Laws


In this case, we see the result is ``NON_ROBUST_MULTI_AGENT_FAIL``, which indicates the agents interfered with each other - specifically, the action ``f1_a2_move(se, sw)`` indicates that agent 2 violated precondition number 1 (the second one - ``free(l2)``), and crashed into another agent.

To remedy that, we can use the waitfor mechanism, to ensure each agent waits until its destination location is clear before moving into it.

Instead of manually modifying the multi agent problem, we will utilize the ``SocialLaw`` class, which represents a modification to a multi-agent problem. 
``SocialLaw`` is implemented as a compiler, which takes a multi-agent problem, and output a modified multi-agent problem.

In [ ]:
from social_laws.social_law import SocialLaw

l = SocialLaw()

for agent in problem.agents:
    # Use only strings because this should work on any given problem, and note the extra comment to make ("l2",) a tuple (as ("l2") is interpreted as "l2" )
    l.add_waitfor_annotation(agent.name, "move", "free", ("l2",)) 
    
new_problem = l.compile(problem).problem

print(new_problem)

problem name = sl_simple_example

types = [loc]

environment fluents = [
  bool connected[l1=loc, l2=loc]
  bool free[l=loc]
]

agents = [
  Agent name = a1

fluents = [
 bool at[l1=loc]
]

actions = [
 action move(loc l1, loc l2) {
    preconditions = [
      at(l1)
      free(l2)
      connected(l1, l2)
    ]
    effects = [
      at(l2) := true
      free(l2) := false
      at(l1) := false
      free(l1) := true
    ]
    simulated effect = None
  }
]


  Agent name = a2

fluents = [
 bool at[l1=loc]
]

actions = [
 action move(loc l1, loc l2) {
    preconditions = [
      at(l1)
      free(l2)
      connected(l1, l2)
    ]
    effects = [
      at(l2) := true
      free(l2) := false
      at(l1) := false
      free(l1) := true
    ]
    simulated effect = None
  }
]


]

objects = [
  loc: [nw, ne, sw, se]
]

initial values = [
  connected(nw, ne) := true
  connected(nw, sw) := true
  connected(ne, nw) := true
  connected(ne, se) := true
  connected(sw, se) := true
  connected(sw, 

We can now call the robustness verification procedure again, to see whether the system (with the new waitfor conditions) is robust.

In [ ]:
result = slrc.is_robust(new_problem)

print(result.status)
print(result.counter_example)

SocialLawRobustnessStatus.NON_ROBUST_MULTI_AGENT_DEADLOCK
[s_a2_move(se, ne), w0_a1_move(nw, ne), pw_a1_move(ne, nw), pw_a1_move(nw, sw), end_f_a1_0, end_s_a2]


The result now is a deadlock (``NON_ROBUST_MULTI_AGENT_DEADLOCK``), caused by agent 1 trying to drive into agent 2's goal, after agent 2 is already there.

We can keep trying to manually add to the social law, but instead we will automatically synthesize a robust social law.

## Automatically Synthesizing Robust Social Laws

The ``SocialLawGenerator`` searches through the space of possible social laws (trying to disallow grounded actions), until it finds a social law which is robust.
It supports various search algorithms and heuristics, but the ``get_gbfs_social_law_generator`` method generates a greedy best first search with the available heuristics.

Calling ``SocialLawGenerator.generate_social_law`` returns a social law which is robust (or ``None`` if it can not find one)

In [ ]:
from social_laws.synthesis import SocialLawGenerator, get_gbfs_social_law_generator

generator = get_gbfs_social_law_generator()
robust_sl = generator.generate_social_law(problem)

print(robust_sl)



added_waitfors: set()
disallowd actions: {('a1', 'move', ('nw', 'ne')), ('a2', 'move', ('se', 'sw'))}
new fluents: set()
new fluents initial vals: set()
added action parameters: set()
added preconditions: set()
new objects: set()


In this case, the social law forbids agent 2 from going from se to sw, and agent1 from going from nw to ne.
We can apply this social law to get a robust multi agent problem.

In [ ]:
robust_problem = robust_sl.compile(problem).problem
print(robust_problem)

problem name = sl_simple_example

types = [loc]

environment fluents = [
  bool connected[l1=loc, l2=loc]
  bool free[l=loc]
]

agents = [
  Agent name = a1

fluents = [
 bool at[l1=loc]
 bool allowed__move[l1=loc, l2=loc]
]

actions = [
 action move(loc l1, loc l2) {
    preconditions = [
      at(l1)
      free(l2)
      connected(l1, l2)
      allowed__move(l1, l2)
    ]
    effects = [
      at(l2) := true
      free(l2) := false
      at(l1) := false
      free(l1) := true
    ]
    simulated effect = None
  }
]


  Agent name = a2

fluents = [
 bool at[l1=loc]
 bool allowed__move[l1=loc, l2=loc]
]

actions = [
 action move(loc l1, loc l2) {
    preconditions = [
      at(l1)
      free(l2)
      connected(l1, l2)
      allowed__move(l1, l2)
    ]
    effects = [
      at(l2) := true
      free(l2) := false
      at(l1) := false
      free(l1) := true
    ]
    simulated effect = None
  }
]


]

objects = [
  loc: [nw, ne, sw, se]
]

initial values = [
  connected(nw, ne) := true


## Planning using robust multi-agent systems

Now that we have a robust multi-agent system, we can be assured that our simple planning strategy of planning for each agent alone, and then combining the plans, is guaranteed to work. This can be done using the same ``SocialLawRobustnessChecker``, which is one a OneShotPlanner

In [ ]:
presult = slrc.solve(robust_problem)

print(presult.status)
print(presult.plan)

PlanGenerationResultStatus.SOLVED_SATISFICING
[a1.move(nw, sw), a2.move(se, ne)]
